In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [11]:
admissions = pd.read_csv('demo_data/ADMISSIONS.csv.gz', compression = 'gzip')
noteevents = pd.read_csv('demo_data/NOTEEVENTS.csv.gz', compression = 'gzip')

/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_81894/1709349325.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents = pd.read_csv('demo_data/NOTEEVENTS.csv.gz', compression = 'gzip')


In [14]:
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

admissions = admissions.sort_values(['SUBJECT_ID','ADMITTIME'])
admissions = admissions.reset_index(drop = True)
admissions['NEXT_ADMITTIME'] = admissions.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
admissions['NEXT_ADMISSION_TYPE'] = admissions.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

rows = admissions.NEXT_ADMISSION_TYPE == 'ELECTIVE'
admissions.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
admissions.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

admissions = admissions.sort_values(['SUBJECT_ID','ADMITTIME'])

#When we filter out the "ELECTIVE", we need to correct the next admit time for these admissions since there might be 'emergency' next admit after "ELECTIVE"
admissions[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admissions.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
admissions['DAYS_NEXT_ADMIT']=  (admissions.NEXT_ADMITTIME - admissions.DISCHTIME).dt.total_seconds()/(24*60*60)
admissions['OUTPUT_LABEL'] = (admissions.DAYS_NEXT_ADMIT < 30).astype('int')
### filter out newborn and death
admissions = admissions[admissions['ADMISSION_TYPE']!='NEWBORN']
admissions = admissions[admissions.DEATHTIME.isnull()]
admissions['DURATION'] = (admissions['DISCHTIME']-admissions['ADMITTIME']).dt.total_seconds()/(24*60*60)

/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_81894/1421443384.py:17: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  admissions[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admissions.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_81894/1421443384.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  admissions[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admissions.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_81894/1421443384.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .f

In [15]:
noteevents = noteevents.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])
adm_notes = pd.merge(admissions[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME','OUTPUT_LABEL','DURATION']],
                     noteevents[['SUBJECT_ID','HADM_ID','CHARTDATE','TEXT','CATEGORY']], 
                     on = ['SUBJECT_ID','HADM_ID'],
                     how = 'left')

In [16]:
adm_notes.ADMITTIME_C = adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])
adm_notes['ADMITTIME_C'] = pd.to_datetime(adm_notes.ADMITTIME_C, format = '%Y-%m-%d', errors = 'coerce')
adm_notes['CHARTDATE'] = pd.to_datetime(adm_notes.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')

### If Discharge Summary 
discharge = adm_notes[adm_notes['CATEGORY'] == 'Discharge summary']
# multiple discharge summary for one admission -> after examination -> replicated summary -> replace with the last one 
discharge = (discharge.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()
discharge = discharge[discharge['TEXT'].notnull()]

### If Less than n days on admission notes (Early notes)
def less_n_days_data (adm_notes, n):
    
    less_n = adm_notes[((adm_notes['CHARTDATE']-adm_notes['ADMITTIME_C']).dt.total_seconds()/(24*60*60))<n]
    less_n=less_n[less_n['TEXT'].notnull()]
    #concatenate first
    concat = pd.DataFrame(less_n.groupby('HADM_ID')['TEXT'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
    concat['OUTPUT_LABEL'] = concat['HADM_ID'].apply(lambda x: less_n[less_n['HADM_ID']==x].OUTPUT_LABEL.values[0])

    return concat

less_2 = less_n_days_data(adm_notes, 2)
less_3 = less_n_days_data(adm_notes, 3)

/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_81894/2455160033.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  adm_notes.ADMITTIME_C = adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])


In [24]:
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    return y

def preprocessing(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))

    df_len = len(df_less_n)
    want=pd.DataFrame({'ID':[],'TEXT':[],'Label':[]})
    for i in tqdm(range(df_len)):
        x=df_less_n.TEXT.iloc[i].split()
        n=int(len(x)/318)
        for j in range(n):
            new_row = pd.DataFrame({'TEXT':' '.join(x[j*318:(j+1)*318]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]}, index=[0])
            want=pd.concat([want, new_row],ignore_index=True)
        if len(x)%318>10:
            new_row = pd.DataFrame({'TEXT':' '.join(x[-(len(x)%318):]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]}, index=[0])
            want=pd.concat([want, new_row],ignore_index=True)
    
    return want

In [25]:
discharge = preprocessing(discharge)
less_2 = preprocessing(less_2)
less_3 = preprocessing(less_3)

100%|██████████| 44551/44551 [35:14<00:00, 21.06it/s]  


In [30]:
less_2.head()

,ID,TEXT,Label
0,100001.0,11:12 am chest (pa & lat) clip # reason: eval ...,0.0
1,100003.0,sinus rhythm prolonged qt interval is nonspeci...,0.0
2,100003.0,"cervical wnl cardiovascular: (s1: normal), (s2...",0.0
3,100003.0,gwp fri 3:58 pm no tapable ascites pv patent w...,0.0
4,100003.0,no pneumothorax or pleural effusion. patient/t...,0.0


In [29]:
adm_notes.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DAYS_NEXT_ADMIT,NEXT_ADMITTIME,ADMISSION_TYPE,DEATHTIME,OUTPUT_LABEL,DURATION,CHARTDATE,TEXT,CATEGORY,ADMITTIME_C
0,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,NaT,EMERGENCY,NaT,0,10.784722,2101-10-20,Sinus rhythm\nInferior/lateral T changes are n...,ECG,2101-10-20
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,NaT,EMERGENCY,NaT,0,10.784722,2101-10-20,[**2101-10-20**] 10:23 PM\n CHEST (PORTABLE AP...,Radiology,2101-10-20
2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,NaT,EMERGENCY,NaT,0,10.784722,2101-10-20,[**2101-10-20**] 6:16 PM\n CHEST (PORTABLE AP)...,Radiology,2101-10-20
3,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,NaT,EMERGENCY,NaT,0,10.784722,2101-10-20,[**2101-10-20**] 5:49 PM\n CT ABDOMEN W/O CONT...,Radiology,2101-10-20
4,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,NaT,EMERGENCY,NaT,0,10.784722,2101-10-21,PATIENT/TEST INFORMATION:\nIndication: S/P Car...,Echo,2101-10-20
